# Phase 2 Polygon API Smoke Test

This notebook tests the Polygon.io integration and options data pipeline.

In [ ]:
import sys
import os
from pathlib import Path

# Add src to path
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from config.settings import POLYGON_API_KEY, DATA_PROVIDER
from data.providers.polygon_client import PolygonClient
from phase2.options_pipeline import OptionsAnalysisPipeline
from common.types import LaunchEvent
from datetime import datetime, date

print(f"Using provider: {DATA_PROVIDER}")
print(f"API Key configured: {'Yes' if POLYGON_API_KEY else 'No'}")

## Test 1: Basic Connection

In [ ]:
# Test basic connection
client = PolygonClient(POLYGON_API_KEY)
is_connected = client.validate_connection()
print(f"Connection successful: {is_connected}")

## Test 2: Stock Data Retrieval

In [ ]:
# Test stock data retrieval
from datetime import timedelta

end_date = date.today() - timedelta(days=1)  # Yesterday
start_date = end_date - timedelta(days=7)    # Last week

stock_data = client.get_stock_data("AAPL", start_date, end_date)
print(f"Retrieved {len(stock_data)} days of AAPL data")
print(stock_data.head())

## Test 3: Options Chain Data

In [ ]:
# Test options chain data
options_date = date.today() - timedelta(days=1)  # Yesterday

options_chain = client.get_options_chain("AAPL", options_date)
print(f"Retrieved {len(options_chain)} options contracts")
if len(options_chain) > 0:
    print("Columns:", options_chain.columns.tolist())
    print(options_chain.head())

## Test 4: Options Trades (Premium Feature)

In [ ]:
# Test options trades (may require premium access)
trades_end = date.today() - timedelta(days=1)
trades_start = trades_end - timedelta(days=3)

options_trades = client.get_options_trades("AAPL", trades_start, trades_end)
print(f"Retrieved {len(options_trades)} options trades")
if len(options_trades) > 0:
    print("Columns:", options_trades.columns.tolist())
    print(options_trades.head())

## Test 5: Full Pipeline Test

In [ ]:
# Test the full options pipeline with a recent event
pipeline = OptionsAnalysisPipeline()

# Create a test event (use a recent Apple event)
test_event = LaunchEvent(
    name="Test iPhone Event",
    company="Apple",
    ticker="AAPL",
    announcement=date(2023, 9, 12),  # iPhone 15 announcement
    release=date(2023, 9, 22),
    next_earnings=date(2023, 11, 2),
    category="Consumer Hardware"
)

print(f"Testing pipeline with: {test_event.name}")
options_data = pipeline.get_options_data_for_event(test_event, lookback_days=5, lookforward_days=2)

print(f"\nChain data: {len(options_data['chain'])} contracts")
print(f"Trades data: {len(options_data['trades'])} trades")

## Test 6: Volume Analysis

In [ ]:
# Test volume analysis if we have trades data
if len(options_data['trades']) > 0:
    analysis = pipeline.analyze_options_volume_patterns(
        options_data['trades'], 
        test_event.announcement
    )
    
    print("Volume Analysis Results:")
    for key, value in analysis.items():
        if key != 'daily_summary':  # Skip the detailed DataFrame
            print(f"  {key}: {value}")
else:
    print("No trades data available for volume analysis")

## Summary

This notebook tests:
1. ✅ API connection
2. ✅ Stock data retrieval
3. ✅ Options chain data
4. ⚠️ Options trades (may require premium)
5. ✅ Pipeline integration
6. ✅ Volume analysis

**Note**: Options trades data typically requires a premium Polygon.io subscription. The free tier provides options chain snapshots but limited historical trades data.